In [1]:
from pytorch_lightning.loggers import WandbLogger
import wandb

wandb.login()
wandb_logger = WandbLogger(
    project='CLIP',
    log_model='all',
    name='CLIP_fold',
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nyugen. Use `wandb login --relogin` to force relogin


In [2]:
from typing import List

import torch as th
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, Callback
# from sklearn.model_selection import train_test_split
import pandas as pd

import src.classification as lc

from pytorch_lightning.callbacks import ModelCheckpoint, Callback, EarlyStopping

from src.experiments import ExperimentMosquitoClassifier

def callbacks() -> List[Callback]:
    return [
        ModelCheckpoint(
            dirpath= "/home/pc2/Downloads/AI-Crowd/Mosquito-Classifiction/experiments/checkpoints/CLIP_folds/f1/",
            monitor="val_f1_score",
            mode="max",
            save_top_k=2,
            save_last=False,
            save_weights_only=True,
            filename="{epoch}-{val_loss}-{val_f1_score}-{val_multiclass_accuracy}",
        ),
        EarlyStopping(monitor="val_f1_score", mode="max", patience=5),
        # LogPredictionsCallback(),
    ]

CLASS_DICT = {
    "albopictus":           th.tensor([1, 0, 0, 0], dtype=th.float),
    "culex":                th.tensor([0, 1, 0, 0], dtype=th.float),
    "japonicus/koreicus":   th.tensor([0, 0, 1, 0], dtype=th.float),
    "culiseta":             th.tensor([0, 0, 0, 1], dtype=th.float),
}

class_dict = {
    "albopictus":           th.tensor([1, 0, 0, 0, 0], dtype=th.float),
    "culex":                th.tensor([0, 1, 0, 0, 0], dtype=th.float),
    "japonicus/koreicus":   th.tensor([0, 0, 1, 0, 0], dtype=th.float),
    "culiseta":             th.tensor([0, 0, 0, 1, 0], dtype=th.float),
    "mosquito":             th.tensor([0, 0, 0, 0, 1], dtype=th.float)
} 

dataset = 'datacomp_xl_s13b_b90k'
aug = 'hca'
bs = 16
img_size = (224, 224)
# img_size = (299, 299)
shift_box = False

img_dir = ""

# Anno 2: F1
f = 1
val_annotations_csv = f"../data_round_2/Folds/val_fold_{f}.csv"
train_annotations_csv = f"../data_round_2/Folds/train_fold_{f}.csv"
test_annotations_csv = f"../data_round_2/mosAlert_new_annotation_2/test_annotation_2.csv"


val_df = pd.read_csv(val_annotations_csv)
train_df = pd.read_csv(train_annotations_csv)
test_df = pd.read_csv(test_annotations_csv)
test_df = test_df.sample(frac=1).reset_index(drop=True) # need shuffle test set


train_dataloader, _, _ = ExperimentMosquitoClassifier(".", "",
                                                       class_dict=CLASS_DICT,
                                                       class_dict_test=class_dict).get_dataloaders(
    train_df,
    val_df,
    test_df,
    dataset,
    aug,
    bs,
    img_size,
    shift_box,
)

_, val_dataloader, _ = ExperimentMosquitoClassifier(img_dir, "",
                                                       class_dict=CLASS_DICT,
                                                       class_dict_test=class_dict).get_dataloaders(
    train_df,
    val_df,
    test_df,
    dataset,
    aug,
    bs,
    img_size,
    shift_box,
)

model = lc.MosquitoClassifier(
    bs=16,
    head_version=7,
    freeze_backbones=False,
    label_smoothing=0.1,
    data_aug="hca",
    epochs=15,
    max_steps=60000,
    use_ema=True,
    n_classes=4,
    loss_func="ce",
    # model_name="hf-hub:imageomics/bioclip",
    #dataset="",
)


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.16 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations
/home/pc2/miniconda3/envs/yolo/lib/python3.12/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/home/pc2/miniconda3/envs/yolo/lib/python3.12/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected
  Expected `Union[float

In [3]:
val_df["class_label"].unique()

array(['japonicus/koreicus', 'culiseta', 'culex', 'albopictus'],
      dtype=object)

In [4]:
for batch in train_dataloader:
    print(batch[0].shape)
    print(batch[1])
    break

torch.Size([16, 3, 224, 224])
tensor([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.]])


In [5]:
# hca_callbacks = callbacks()
th.set_float32_matmul_precision("high")
trainer = pl.Trainer(
    logger=wandb_logger,
    accelerator="gpu",
    precision="16-mixed",
    max_epochs=model.hparams.epochs,
    # logger=True,
    deterministic=True,  # maybe we should add this
    callbacks=callbacks(),
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
import torch
torch.cuda.empty_cache()


In [7]:
trainer.fit(
    model=model,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type           | Params | Mode 
------------------------------------------------
0 | cls  | CLIPClassifier | 303 M  | train
1 | ema  | EMA            | 303 M  | train
------------------------------------------------
607 M     Trainable params
0         Non-trainable params
607 M     Total params
2,431.767 Total estimated model params size (MB)
278       Modules in train mode
277       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/pc2/miniconda3/envs/yolo/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/pc2/miniconda3/envs/yolo/lib/python3.12/site-packages/torcheval/metrics/functional/classification/accuracy.py:275: UserWarning: The reduce argument of torch.scatter with Tensor src is deprecated and will be removed in a future PyTorch release. Use torch.scatter_reduce instead for more reduction options. (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:231.)
  num_correct = mask.new_zeros(num_classes).scatter_(0, target, mask, reduce="add")
/home/pc2/miniconda3/envs/yolo/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consid

Training: |          | 0/? [00:00<?, ?it/s]

/home/pc2/miniconda3/envs/yolo/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=15` reached.


In [8]:
wandb.finish() 

epoch,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇███
train_accuracy,▁▆▆▇▇▇▇████████
train_f1_score,▁▆▆▇▇▇▇█▇▇█████
train_loss,█▆▄▄█▆▅▅▄▅▄▆▅▂▇▁▄▅▃▃▄▂▄▃▆▅▄▃▄▃▄▃▅▄▄▃▆▆▅▄
train_multiclass_accuracy,▁▆▆▇▇▇▇████████
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
val_accuracy,▁▅▆▇▇▇▇▇██▇██▇█
val_f1_score,▁▅▇▇▇▇▇▇▇▇▇█▇▇█
val_loss,█▃▂▁▁▂▁▁▁▁▁▁▁▁▁
val_multiclass_accuracy,▁▇█▆▆▆▇▅▅▅▄▄▄▄▅
epoch,14
